In [1]:
from pyspark.sql import functions as F, SparkSession, Window
import pandas as pd

In [2]:
#  Create a Spark Session
spark = (
    SparkSession.builder.appName("Ranking Model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)

24/09/30 22:03:19 WARN Utils: Your hostname, DESKTOP-H6V94HM resolves to a loopback address: 127.0.1.1; using 192.168.0.236 instead (on interface wifi0)
24/09/30 22:03:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/30 22:03:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Feature engineering

In [3]:
transactions = spark.read.parquet(f"../data/curated/transactions.parquet")
transactions = transactions.withColumn("order_month", F.date_format(F.col("order_datetime"), "yyyy-MM"))

In [4]:
transactions.limit(1)

merchant_abn,order_datetime,consumer_id,dollar_value,order_id,consumer_fp,merchant_fp,name,category,revenue_level,take_rate,order_month
19854089605,2021-08-19,285333,244.11185528431417,ab5d50f5-cf77-47f...,NULL,NULL,Aenean Massa Indu...,"gift, card, novel...",b,3.15,2021-08


We think that the forecasted revenue of each month for each merchant will be useful in determining if we should onboard the merchant to the BNPL firm as it's an indication of future growth. The growth in number of orders each month might also be important.

In [5]:
# Aggregate the data into desired format
agg_transactions = transactions.groupBy("merchant_abn", "order_month", "take_rate").agg(
    F.count(F.col("order_id")).alias("num_orders"),
    F.round(F.sum('dollar_value'), 2).alias('revenue'),
    F.round(F.mean(F.col("dollar_value")),2).alias("revenue_per_order")
)
agg_transactions = agg_transactions.orderBy(["merchant_abn", "order_month"], ascending = [False, True])

# Partition the data based on merchant ABN and compute lag variables for each specific partition
merchant_partition = Window.partitionBy("merchant_abn").orderBy("order_month")
agg_transactions = agg_transactions.withColumns({
    "revenue_lag": F.lag("revenue").over(merchant_partition),
    "expected_profit": F.round(F.col("revenue") * F.col("take_rate")/100,2)
    })
agg_transactions = agg_transactions.withColumn("revenue_growth",
                                               F.when(F.col("revenue_lag").isNotNull(), F.round((F.col("revenue") - F.col("revenue_lag"))/F.col("revenue_lag"), 2)).otherwise(None))

We originally planned to use features such as firm value's using the Discounted Cash Flow model - assuming revenue is cash flow - and the Internal Rate of Return model. However, some merchants don't have consistent sales periods i.e. some sales periods are missing for some merchants. This situation would make the features unreliable and the comparision between merchants harder. Thus, we decide to assume that the missing sales records for periods are totally due to the merchant unable to make any sales during those period and is not because of data entry errors. We understand that there might be some merchants that only do business during certain period but looking from a perspective of a business, it's better to have merchants that have consistent sales.

In [6]:
# Create a dataframe that store the valid date range of the data
months = pd.date_range(start="2021-05-01", end="2022-08-31", freq='MS').strftime('%Y-%m').tolist()
months_df = spark.createDataFrame([(month,) for month in months], ["order_month"])

# Join with the transactions dataframe
transactions_correct_months = transactions.join(months_df, on= ['order_month'], how = 'right')

# Get the list of merchants with complete sales records
complete_merchants = transactions_correct_months.groupBy("merchant_abn").agg(F.countDistinct("order_month")\
                                                                             .alias("month_count")).filter(F.col("month_count") == len(months)).select("merchant_abn")
print(f"Number of merchants with complete sales records: {complete_merchants.count()}")

# Select the merchants with complete sales records from the aggregated sales 
print(f"Number of entries before removing merchants with missing sales records: {agg_transactions.count()}")
agg_transactions = agg_transactions.join(complete_merchants, on='merchant_abn', how='inner')
agg_transactions = agg_transactions.filter("2021-05" <= F.col("order_month")).orderBy(['merchant_abn', 'order_month'],
                                                                                      ascending = [True, True])
print(f"Number of entries after removing merchants with missing sales records: {agg_transactions.count()}")

24/09/30 22:03:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:03:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:03:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Number of merchants with complete sales records: 3194


Number of entries before removing merchants with missing sales records: 67331


Number of entries after removing merchants with missing sales records: 51104


In [7]:
agg_transactions

24/09/30 22:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:04:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:04:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:04:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:04:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,order_month,take_rate,num_orders,revenue,revenue_per_order,revenue_lag,expected_profit,revenue_growth
10023283211,2021-05,0.18,144,30111.88,209.11,27622.34,54.2,0.09
10023283211,2021-06,0.18,145,28790.43,198.55,30111.88,51.82,-0.04
10023283211,2021-07,0.18,153,29430.73,192.36,28790.43,52.98,0.02
10023283211,2021-08,0.18,146,32118.49,219.99,29430.73,57.81,0.09
10023283211,2021-09,0.18,175,34755.78,198.6,32118.49,62.56,0.08
10023283211,2021-10,0.18,169,39993.29,236.65,34755.78,71.99,0.15
10023283211,2021-11,0.18,217,48435.09,223.2,39993.29,87.18,0.21
10023283211,2021-12,0.18,211,45068.95,213.6,48435.09,81.12,-0.07
10023283211,2022-01,0.18,128,29431.27,229.93,45068.95,52.98,-0.35
10023283211,2022-02,0.18,134,31489.46,235.0,29431.27,56.68,0.07


In [10]:
merchant_avg_growth = agg_transactions.groupBy("merchant_abn").agg(
    F.round(F.mean("revenue_growth"),2).alias("avg_growth"),
    F.round(F.mean("expected_profit"), 2).alias('avg_monthly_expected_profit'),
    F.round(F.mean("revenue_per_order"),2).alias('avg_monthly_revenue_per_order'),
    F.round(F.mean('num_orders'), 2).alias('avg_monthly_num_order')
).orderBy("merchant_abn", ascending=True)

In [11]:
merchant_avg_growth

24/09/30 22:07:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:07:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/30 22:07:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


merchant_abn,avg_growth,avg_monthly_expected_profit,avg_monthly_revenue_per_order,avg_monthly_num_order
10023283211,0.02,63.71,216.01,164.06
10142254217,0.05,248.76,40.01,147.56
10187291046,0.05,64.28,120.95,16.38
10192359162,0.19,534.56,444.36,18.88
10206519221,0.03,1129.8,37.85,469.31
10255988167,0.12,702.47,389.96,41.63
10264435225,0.04,700.22,116.12,251.75
10279061213,0.07,481.16,297.37,28.31
10323485998,0.04,4453.99,128.58,524.13
10342410215,0.02,1045.22,386.85,43.0
